# NLOptControl.jl

## Installation

There are several packages that need to be installed, to do this run:
```julia
Pkg.clone("https://github.com/JuliaMPC/PrettyPlots.jl")
Pkg.clone("https://github.com/JuliaMPC/VehicleModels.jl")
Pkg.clone("https://github.com/JuliaMPC/NLOptControl.jl")
```

Either [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) or [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) can be used for the nonlinear solver. Since **Ipopt** is free, all of the examples will use it and it can be added with:
```julia
Pkg.add("Ipopt");Pkg.build("Ipopt");
```

If you are using **Linux** make sure that you have **gfortran** to run **Ipopt**:
```julia
sudo apt-get install gfortran
```

Also, a plotting backend will be required and there are several options:
```julia
Pkg.add("PGFPlots");Pkg.build("PGFPlots")   # best looking
Pkg.add("GR");Pkg.build("GR");              # most reliable
Pkg.add("PyPlot");Pkg.build("PyPlot")       # also a good option  
```

## Packages that will be used

In [1]:
using NLOptControl, PrettyPlots
#using Plots; pgfplots();           # this is optional, by default PrettyPlots uses gr() as a backend

INFO: Recompiling stale cache file /home/febbo/.julia/lib/v0.6/VehicleModels.ji for module VehicleModels.


## Quick Example: Brachistochrone
#### Solved by: John and Bernoulli, Newton and L'Hospital
### Given:
#### A particle sliding without friction along an unknown track in a gravitational feild
#### Dynamic Constraints
$$\dot{x}_1(t)=x_3(t)sin(u_1(t))$$ 
$$\dot{x}_2(t)=-x_3(t)cos(u_1(t))$$
$$\dot{x}_3(t)=gcos(u_1(t))$$

#### Boundary Conditions
$${x}_1(0)=0 \qquad {x}_1(t_f)=2$$
$${x}_2(0)=0\qquad {x}_2(t_f)=-2$$
$${x}_3(0)=0\qquad {x}_3(t_f)=free$$
### Find:
#### The track that minimizes time
$$J=t_f$$
### Solution: 

#### Differential Equations

Now we need to write all of the given information out. Let's start with the differential equation, that is written as an array of expressions:

In [2]:
de=[:(x3[j]*sin(u1[j])),:(-x3[j]*cos(u1[j])),:(9.81*cos(u1[j]))]

3-element Array{Expr,1}:
 :(x3[j] * sin(u1[j]))   
 :(-(x3[j]) * cos(u1[j]))
 :(9.81 * cos(u1[j]))    

Next let's write down the boundary conditions into an array:

In [3]:
X0=[0.0,0.0,0.0]
XF=[2.,-2.,NaN]

3-element Array{Float64,1}:
   2.0
  -2.0
 NaN  

#### Notice:
1. The numbers that where put into the expression are `Float64`; For now this is required!
2. The `NaN` is put into the boundary constraint for the third state; If any of the state bounds are free then pass a `NaN`

#### Define the Problem:
Now that we have the basic problem written down, we can call the `define()` function as:

In [4]:
n=define(de;numStates=3,numControls=1,X0=X0,XF=XF);

#### Basics
Variable | Description
:--- | :--- 
`n` | object that holds the entire optimal control problem 
`de` | array of differential equation expressions
`numStates` | the number of states
`numControls` | the number of controls
`X0` | intial state constraint
`XF` | final state constraint


Also, not in this problem, but

Variable | Description
:--- | :--- 
`XL` | lower state bound
`XU` | upper state bound
`CL` | lower state bound
`CU` | upper control bound

The above bounds can be set in the same fashion as the initial and final state constraints. i.e. in an Array.

#### Configure the Problem:
Now that the basic optimal control problem has been defined, the next step is to `configure!()` with additional options.

In [5]:
configure!(n;(:Nck=>[100]),(:finalTimeDV=>true));

#### Settings:
Key | Description
:--- | :--- 
`:Nck` | array of that holds the number of points within each interval
`:finalTimeDV` | bool to indicate if time is a design variable

#### Notice:
1. Final time is a design variable; we are trying to minimize it
2. We defined this as a single interval problem with ``100`` points

#### Objective Function
Finally, the objective function needs to be defined. For this, we use the ``JuMP`` macro ``@NLOptControl()`` directly as:

In [6]:
@NLobjective(n.mdl,Min,n.tf);

with,

Variable | Description
:--- | :--- 
`n.mdl` | object that holds them JuMP model 
`Min` | for a minimization problem
`n.tf` | a reference to the final time

#### Optimize
Now that the entire optimal control problem has been defined we can `optimize!()` it as:

In [7]:
optimize!(n);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



#### Post Process
**Make sure that you are not running the code in a folder where you have an important folder named `results`, because it will be deleted!**
Now that the problem has been optimized, we can quickly visualize the solution using ``allPlots()`` as:


In [8]:
allPlots(n)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 time (s) 
 
 
 x1 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2.7; stroke-opacity:1; fill:none" points="
 39.3701,552.637 39.683,552.637 40.4189,552.637 41.5747,552.637 43.149,552.637 45.1404,552.637 47.5469,552.636 50.3661,552.636 53.5953,552.634 57.2311,552.631 
 61.2701,552.626 65.7083,552.617 70.5412,552.604 75.7642,552.585 81.372,552.556 87.3591,552.517 93.7197,552.463 100.447,552.389 107.536,552.293 114.977,552.168 
 122.765,552.008 130.891,551.805 139.348,551.554 148.127,551.244 157.219,550.866 166.616,550.409 176.308,549.863 186.286,549.215 196.539,548.453 207.058,547.562 
 217.833,546.529 228.852,545.338 240.104,543.974 251.58,542.422 263.267,540.664 275.153,538.684 287.228,536.467 299.479,533.995 311.894,531.253 324.46,528.225 
 337.167,524.896 350,521.251 362.947,517.277 375.996,512.962 389.133,508.294 402.346,503.263 415.621,497.861 428.946,492.081 442.308,485.918 455.692,479.369 
 469.086,472.433 482.477,465.111 495.852,457.404 509.196,449.32 522.498,440.864 535.744,432.047 548.921,422.879 562.015,413.374 575.015,403.549 587.907,393.421 
 600.678,383.009 613.316,372.334 625.808,361.421 638.143,350.294 650.307,338.979 662.289,327.504 674.077,315.897 685.66,304.188 697.025,292.406 708.163,280.585 
 719.061,268.754 729.709,256.946 740.096,245.192 750.213,233.526 760.049,221.978 769.595,210.581 778.84,199.365 787.777,188.361 796.396,177.599 804.688,167.109 
 812.646,156.918 820.262,147.053 827.528,137.542 834.437,128.408 840.982,119.676 847.156,111.369 852.954,103.507 858.37,96.112 863.399,89.201 868.035,82.7918 
 872.274,76.9004 876.112,71.5413 879.545,66.7275 882.57,62.4708 885.183,58.7813 887.382,55.6679 889.165,53.138 890.53,51.1977 891.477,49.8514 892.003,49.1021 
 892.126,48.9264 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mpc 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 time (s) 
 
 
 x2 
 
<polyline clip-path="url(#clip03)" style="stroke:#0000ff; stroke-width:2.7; stroke-opacity:1; fill:none" points="
 939.37,23.7409 939.683,23.741 940.419,23.7422 941.575,23.7465 943.149,23.7574 945.14,23.7793 947.547,23.8181 950.366,23.8805 953.595,23.9745 957.231,24.1091 
 961.27,24.2944 965.708,24.5413 970.541,24.8618 975.764,25.2686 981.372,25.775 987.359,26.3953 993.72,27.1441 1000.45,28.0366 1007.54,29.0882 1014.98,30.3148 
 1022.77,31.7322 1030.89,33.3563 1039.35,35.203 1048.13,37.2878 1057.22,39.6257 1066.62,42.2313 1076.31,45.1187 1086.29,48.301 1096.54,51.7902 1107.06,55.5975 
 1117.83,59.7328 1128.85,64.2045 1140.1,69.0199 1151.58,74.1843 1163.27,79.7016 1175.15,85.5739 1187.23,91.8015 1199.48,98.3829 1211.89,105.314 1224.46,112.59 
 1237.17,120.204 1250,128.145 1262.95,136.403 1276,144.964 1289.13,153.813 1302.35,162.934 1315.62,172.309 1328.95,181.917 1342.31,191.737 1355.69,201.746 
 1369.09,211.922 1382.48,222.239 1395.85,232.672 1409.2,243.196 1422.5,253.784 1435.74,264.41 1448.92,275.047 1462.02,285.669 1475.01,296.251 1487.91,306.766 
 1500.68,317.19 1513.32,327.499 1525.81,337.67 1538.14,347.682 1550.31,357.513 1562.29,367.145 1574.08,376.56 1585.66,385.741 1597.03,394.674 1608.16,403.344 
 1619.06,411.741 1629.71,419.855 1640.1,427.675 1650.21,435.197 1660.05,442.413 1669.59,449.32 1678.84,455.914 1687.78,462.195 1696.4,468.162 1704.69,473.816 
 1712.65,479.158 1720.26,484.191 1727.53,488.919 1734.44,493.345 1740.98,497.475 1747.16,501.312 1752.95,504.863 1758.3

`allPlots()` automatically plots the solution to all of the state and control variables. In this problem, we may be interested in comparing two states against one another which can be done using the `statePlot()` function as:

In [9]:
st=statePlot(n,1,1,2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 x1 
 
 
 x2 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2.7; stroke-opacity:1; fill:none" points="
 39.3701,382.271 39.3701,382.271 39.3701,382.273 39.3701,382.279 39.3703,382.295 39.3708,382.328 39.3721,382.385 39.3749,382.478 39.3805,382.618 39.3906,382.818 
 39.408,383.094 39.436,383.461 39.4793,383.937 39.5439,384.542 39.6372,385.295 39.7683,386.217 39.9485,387.33 40.1907,388.657 40.5104,390.22 40.9254,392.043 
 41.4562,394.15 42.1259,396.565 42.9602,399.31 43.9881,402.409 45.241,405.884 46.7535,409.758 48.5629,414.05 50.7095,418.781 53.2359,423.968 56.1874,429.627 
 59.6116,435.775 63.5579,442.422 68.0777,449.58 73.2237,457.258 79.0495,465.459 85.6094,474.189 92.9578,483.447 101.149,493.23 110.236,503.534 120.271,514.35 
 131.304,525.668 143.383,537.473 156.552,549.749 170.854,562.475 186.324,575.631 202.996,589.19 220.899,603.125 240.053,617.408 260.476,632.005 282.179,646.885 
 305.165,662.012 329.432,677.349 354.97,692.859 381.762,708.503 409.784,724.243 439.005,740.039 469.387,755.851 500.885,771.642 533.446,787.371 567.012,803.003 
 601.517,818.499 636.891,833.824 673.056,848.944 709.931,863.827 747.429,878.442 785.458,892.76 823.924,906.756 862.729,920.404 901.771,933.683 940.949,946.572 
 980.156,959.055 1019.29,971.115 1058.24,982.741 1096.9,993.922 1135.17,1004.65 1172.94,1014.92 1210.11,1024.72 1246.58,1034.06 1282.24,1042.93 1317.01,1051.33 
 1350.78,1059.27 1383.47,1066.76 1414.99,1073.78 1445.26,1080.36 1474.2,1086.5 1501.73,1092.21 1527.78,1097.49 1552.29,1102.35 1575.19,1106.8 1596.43,1110.85 
 1615.96,1114.51 1633.72,1117.79 1649.67,1120.69 1663.78,1123.22 1676,1125.39 1686.32,1127.2 1694.7,1128.66 1701.13,1129.77 1705.6,1130.54 1708.08,1130.96 
 1708.66,1131.06 
 "/>
 
 
 
 
 mpc

For this case, there are four things that need to be passed to `statePlots()`:

Argument | Name | Description
:--- | :--- | :---
1|`n` |object that holds the entire optimal control problem
2|`idx` | reference to solution number used when we start solving `mpc` problems
3|`st1` | state number for `xaxis`
4|`st2` | state number for `yaxis`

#### Data Orginization
All of the states, control variables and time vectors are stored in an array of Dataframes called `n.r.dfs`

In [10]:
n.r.dfs

1-element Array{Any,1}:
 101×5 DataFrames.DataFrame
│ Row │ t           │ x1           │ x2           │ x3         │ u1          │
├─────┼─────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ 1   │ 0.0         │ -1.92482e-28 │ 0.0          │ 0.0        │ 3.7389e-19  │
│ 2   │ 0.000302536 │ 1.32471e-10  │ -4.48945e-7  │ 0.00296788 │ 0.000442609 │
│ 3   │ 0.0010139   │ 4.98629e-9   │ -5.04231e-6  │ 0.00994636 │ 0.00148333  │
│ 4   │ 0.00213113  │ 4.63039e-8   │ -2.2277e-5   │ 0.0209063  │ 0.00311783  │
│ 5   │ 0.00365302  │ 2.33209e-7   │ -6.54545e-5  │ 0.035836   │ 0.00534436  │
│ 6   │ 0.00557807  │ 8.30305e-7   │ -0.000152615 │ 0.0547203  │ 0.00816071  │
│ 7   │ 0.00790437  │ 2.36255e-6   │ -0.000306446 │ 0.0775401  │ 0.0115641   │
│ 8   │ 0.0106296   │ 5.74545e-6   │ -0.000554166 │ 0.104272   │ 0.0155511   │
│ 9   │ 0.0137511   │ 1.24386e-5   │ -0.00092738  │ 0.13489    │ 0.0201179   │
│ 10  │ 0.0172658   │ 2.46206e-5   │ -0.00146191  │ 0.16936    │ 0.0252599   │


It is an array because the problem is designed to be solved multiple times in a receding time horizon. The variables can be accesed like this:

In [11]:
n.r.dfs[1][:x1][1:4]

4-element DataArrays.DataArray{Float64,1}:
 -1.92482e-28
  1.32471e-10
  4.98629e-9 
  4.63039e-8 

#### Optimization Data

In [12]:
n.r.dfs_opt

1-element Array{Any,1}:
 1×4 DataFrames.DataFrame
│ Row │ t_solve │ obj_val  │ status  │ iter_num │
├─────┼─────────┼──────────┼─────────┼──────────┤
│ 1   │ 6.06117 │ 0.824339 │ Optimal │ 0        │

The sailent optimization data is stored in the table above

Variable | Description
:--- | :--- 
`t_solve` | cpu time for optimization problem
`obj_val` | objective function value
`iter_num` | a variable for a higher-level algorithm, often these problems are nested

One thing that may be noticed is the long time that it takes to solve the problem. This is typical for the first optimization, but after that even if the problem is modified the optimization time is greatly reduced. 

#### For instance, let's re-run the optimization:

In [13]:
optimize!(n);

In [14]:
n.r.dfs_opt[2][:t_solve]

1-element DataArrays.DataArray{Float64,1}:
 0.1824

#### Since the problem is hot-started it can solve much faster!

## Quick Example: Moon Lander

### Given:
#### A space-ship landing on the moon 
#### Dynamic Constraints
$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=u(t)-g$$

#### Boundary Conditions
$${x}_1(0)=10 \qquad {x}_1(t_f)=0$$
$${x}_2(0)=-2\qquad {x}_2(t_f)=0$$

#### Control Limits
$${u}_{1_{min}}=0 $$
$${u}_{1_{max}}=3 $$

### Find:
#### The track that minimizes time
$$J=\int_{0}^{tf} u(t) dt$$
### Solution:
In this problem, we put the bounds directly into `define()`. Also, now we have constant limits on the control variables and those can be added as shown below

In [15]:
de=[:(x2[j]),:(u1[j]-1.625)]
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);
configure!(n,Nck=[10,10,10,10];(:finalTimeDV=>true));

#### Optional state and control variable descriptions
The state and control variables are by default, $:x1,:x2,..$ and $:u1,:u2,..$, but they can be changed for the plots, **i.e. n.dfs** with the following commands:

In [16]:
names=[:H,:V]; descriptions = ["Height(t)","Velocity(t)"]; stateNames!(n,names,descriptions);
names=[:T]; descriptions = ["Thrust(t)"]; controlNames!(n,names,descriptions);

Next, now that the problem is configured, all of the state and control variables are stored in JuMP Arrays, `n.r.x[:,:]` and `n.r.u[:,:]`, respectively. For instance;

In [17]:
typeof(n.r.x)

Array{JuMP.Variable,2}

#### Integral Terms in the Cost Function
`integrate!()` is used to make terms that can be added to the cost function that need to be integrated. When calling this function an expression must be passed:

In this example the first control variable `T`needs to be integrated. To do this, `integrate!()` can be used as: 

In [18]:
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));

#### Now this term can be added as the objective function and the problem can be solved

In [19]:
@NLobjective(n.mdl, Min, obj);
optimize!(n);

#### Optional plot settings
Many of the plot settings can be modified using the `plotSettings()` function. For instance;

In [20]:
plotSettings(;(:mpc_lines =>[(4.0,:red,:solid)]),(:size=>(700,700)));
allPlots(n)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 time (s) 
 
 
 Height(t) 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" points="
 39.3701,30.6201 42.1144,32.8102 48.3074,38.0013 57.3218,46.1733 68.2744,57.0851 80.093,70.0689 91.6206,83.9423 101.729,97.0902 109.429,107.722 113.968,114.239 
 115.059,115.833 117.803,119.887 123.996,129.303 133.011,143.483 143.963,160.417 155.782,177.684 167.31,193.528 177.418,206.561 185.118,215.994 189.657,221.323 
 190.748,222.588 193.492,225.718 199.685,232.572 208.7,242.027 219.652,252.683 231.471,263.159 242.999,272.353 253.107,279.584 260.807,284.57 265.346,287.299 
 266.437,287.931 269.181,289.482 275.374,292.772 284.389,297.038 295.341,301.391 307.16,305.064 318.688,307.624 328.796,309.037 336.496,309.592 341.035,309.708 
 342.126,309.713 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mpc 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 time (s) 
 
 
 Velocity(t) 
 
<polyline clip-path="url(#clip03)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" points="
 389.37,155.654 392.114,160.016 398.307,169.861 407.322,184.191 418.274,201.601 430.093,220.389 441.621,238.714 451.729,254.782 459.429,267.022 463.968,274.238 
 465.059,275.972 467.803,280.157 473.996,290.926 483.011,295.758 493.963,282.95 505.782,267.607 517.31,251.549 527.418,238.427 535.118,227.685 539.657,221.86 
 540.748,220.281 543.492,216.59 549.685,208.26 558.7,196.135 569.652,181.402 581.471,165.505 592.999,150 603.107,136.403 610.807,126.046 615.346,119.941 
 616.437,118.473 619.181,114.782 625.374,106.452 634.389,94.3268 645.341,79.5946 657.16,63.6976 668.688,48.192 678.796,34.5955 686.496,24.2386 691.035,18.1327 
 692.126,16.6654 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mpc 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 time (s) 
 
 
 Thrust(t) 
 
<polyline clip-path="url(#clip04)" style="stroke:#ff1493; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 39.3701,380.62 42.4282,380.62 45.4864,380.62 48.5445,380.62 51.6026,380.62 54.6608,380.62 57.7189,380.62 60.7771,380.62 63.8352,380.62 66.8933,380.62 
 69.9515,380.62 73.0096,380.62 76.0678,380.62 79.1259,380.62 82.184,380.62 85.2422,380.62 88.3003,380.62 91.3585,380.62 94.4166,380.62 97.4747,380.62 
 100.533,380.62 103.591,380.62 106.649,380.62 109.707,380.62 112.765,380.62 115.824,380.62 118.882,380.62 121.94,380.62 124.998,380.62 128.056,380.62 
 131.114,380.62 134.172,380.62 137.231,380.62 140.289,380.62 143.347,380.62 146.405,380.62 149.463,380.62 152.521,380.62 155.579,380.62 158.638,380.62 
 161.696,380.62 164.754,380.62 167.812,380.62 170.87,380.62 173.928,380.62 176.986,380.62 180.045,380.62 183.103,380.62 186.161,380.62 189.219,380.62 
 192.277,380.62 195.335,380.62 198.393,380.62 201.452,380.62 204.51,380.62 207.568,380.62 210.626,380.62 213.684,380.62 216.742,380.62 219.8,380.62 
 222.859,380.62 225.917,380.62 228.975,380.62 232.033,380.62 235.091,380.62 238.149,380.62 241.207,380.62 244.265,380.62 247.324,380.62 250.382,380.62 
 253.44,380.62 256.498,380.62 259.556,380.62 262.614,380.62 265.672,380.62 268.731,380.62 271.789,380.62 274.847,380.62 277.905,380.62 280.963,380.62 
 284.021,380.62 287.079,380.62 290.138,380.62 293.196,380.62 296.254,380.62 299.312,380.62 302.37,380.62 305.428,380.62 308.486,380.62 311.545,380.62 
 314.603,380.62 317.661,380.62 320.719,380.62 323.777,3

## Other Dynamic Constraint Methods
Currently there are three different methods to ensure that the dyanamic constraints are satisfied and they are set when `configure!()` is called using the `:integrationScheme` key. They are listed below:

`:integrateScheme` | Description
:--- | :---
`:lgrExplicit`| default scheme; implementation of hp-pseudospecral method
`:bkwEuler` | approximate using backward euler method
`:trapezoidal` | approximate using trapezoidal method

The later two are time-marching methods and default number of points is `100`, but that can be changed by setting `N`. So, the above problem can be solved using one of the time-marching schemes as: 

In [21]:
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);
configure!(n,N=200;(:integrationScheme=>:trapezoidal),(:finalTimeDV=>true));
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));
@NLobjective(n.mdl, Min, obj);
optimize!(n);
allPlots(n)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 time (s) 
 
 
 x1 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" points="
 39.3701,30.6201 40.8839,31.8195 42.3976,33.0396 43.9114,34.2803 45.4252,35.5415 46.939,36.8233 48.4528,38.1258 49.9665,39.4487 51.4803,40.7923 52.9941,42.1565 
 54.5079,43.5413 56.0217,44.9466 57.5354,46.3725 59.0492,47.819 60.563,49.2861 62.0768,50.7738 63.5906,52.282 65.1043,53.8109 66.6181,55.3603 68.1319,56.9303 
 69.6457,58.5209 71.1594,60.1321 72.6732,61.7639 74.187,63.4163 75.7008,65.0892 77.2146,66.7827 78.7283,68.4969 80.2421,70.2315 81.7559,71.9868 83.2697,73.7627 
 84.7835,75.5592 86.2972,77.3762 87.811,79.2138 89.3248,81.072 90.8386,82.9508 92.3524,84.8502 93.8661,86.7702 95.3799,88.7107 96.8937,90.6718 98.4075,92.6535 
 99.9213,94.6559 101.435,96.6787 102.949,98.7222 104.463,100.786 105.976,102.871 107.49,104.976 109.004,107.102 110.518,109.248 112.031,111.415 113.545,113.603 
 115.059,115.811 116.573,118.04 118.087,120.289 119.6,122.559 121.114,124.85 122.628,127.161 124.142,129.492 125.656,131.845 127.169,134.217 128.683,136.611 
 130.197,139.024 131.711,141.445 133.224,143.857 134.738,146.252 136.252,148.63 137.766,150.99 139.28,153.333 140.793,155.658 142.307,157.966 143.821,160.257 
 145.335,162.53 146.848,164.785 148.362,167.024 149.876,169.244 151.39,171.448 152.904,173.634 154.417,175.802 155.931,177.953 157.445,180.087 158.959,182.203 
 160.472,184.302 161.986,186.384 163.5,188.448 165.014,190.494 166.528,192.524 168.041,194.535 169.555,196.53 171.069,198.507 172.583,200.466 174.096,202.408 
 175.61,204.333 177.124,206.24 178.638,208.13 180.152,210.003 181.665,211.858 183.179,213.695 184.693,215.515 186.207,217.318 187.72,219.104 189.234,220.872 
 190.748,222.622 192.262,224.355 193.776,226.071 195.289,227.769 196.803,229.45 198.317,231.113 199.831,232.759 201.344,234.388 202.858,235.999 204.372,237.593 
 205.886,239.169 207.4,240.728 208.913,242.27 210.427,243.794 211.941,245.3 213.455,246.79 214.969,248.262 216.482,249.716 217.996,251.153 219.51,252.572 
 221.024,253.975 222.537,255.359 224.051,256.727 225.565,258.077 227.079,259.409 228.593,260.724 230.106,262.022 231.62,263.302 233.134,264.565 234.648,265.81 
 236.161,267.038 237.675,268.249 239.189,269.442 240.703,270.618 242.217,271.776 243.73,272.917 245.244,274.04 246.758,275.146 248.272,276.235 249.785,277.306 
 251.299,278.36 252.813,279.396 254.327,280.415 255.841,281.417 257.354,282.401 258.868,283.368 260.382,284.317 261.896,285.249 263.409,286.163 264.923,287.06 
 266.437,287.94 267.951,288.802 269.465,289.647 270.978,290.474 272.492,291.284 274.006,292.077 275.52,292.852 277.033,293.61 278.547,294.35 280.061,295.073 
 281.575,295.778 283.089,296.466 284.602,297.137 286.116,297.79 287.63,298.426 289.144,299.044 290.657,299.645 292.171,300.228 293.685,300.794 295.199,301.343 
 296.713,301.874 298.226,302.388 299.74,302.885 301.254,303.364 302.768,303.825 304.281,304.269 305.795,304.696 307.309,305.105 308.823,305.497 310.337,305.872 
 311.85,306.229 313.364,306.569 314.878,306.891 316.392,307.196 317.906,307.483 319.419,307.753 320.933,308.006 322.447,308.241 323.961,308.458 325.474,308.659 
 326.988,308.842 328.502,309.007 330.016,309.155 331.53,309.286 333.043,309.399 334.557,309.495 336.071,309.573 337.585,309.634 339.098,309.678 340.612,309.704 
 342.126,309.713 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


## Constraints
Often when building a model and using it to solve an optimal control problem, their are issues associated with infeasibility. `NLOptControl` has functionality to help deal with these issues. For instance, the dual infeasibility values can stored and quickly viewed. They are stored in an array of DataFrames which can be referenced with `n.r.dfs_con` as:  

In [22]:
n.r.dfs_con

1-element Array{Any,1}:
 0×1 DataFrames.DataFrame


It is empty, because by default this data is not calculated and stored. This option can be turned on by modifying the settings for the problem:

In [23]:
n.s.evalConstraints

false

In [24]:
n.s.evalConstraints=true;
optimize!(n);
n.r.dfs_con

2-element Array{Any,1}:
 0×1 DataFrames.DataFrame
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [25]:
evalMaxDualInf(n)

2.183937957922585

The last function called, searches through all of the `dual infeasibilities` to find the largest value. 
As, this problem is, it is feasible and optimal. But if there was an issue, often looking for high values in these DataFrame structures is the quickest way to figure out the constraints that are giving the solver trouble. 
## Tolerances
If there was an example where the `dual infeasibility` value for one or more of the variables was very high, but the actual constraint is only being violated slightly (by some reasonable amount) then the tolerances on the initial and terminal states can be adjusted. This will also improve the solve time, so it is good practice to set these to reasonable values. For instance, in the `Moon Lander` example, we can set them as:

In [26]:
n=define(de;numStates=2,numControls=1,X0=[10.,-2],XF=[0.,0.],CL=[0.],CU=[3.]);

XF_tol=[2.0,0.5];
X0_tol=[0.05,0.05];
defineTolerances!(n;X0_tol=X0_tol,XF_tol=XF_tol);

configure!(n,N=50;(:integrationScheme=>:bkwEuler),(:finalTimeDV=>true));
obj=integrate!(n,n.r.u[:,1];(:variable=>:control));
@NLobjective(n.mdl, Min, obj);
optimize!(n);
allPlots(n) 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 time (s) 
 
 
 x1 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" points="
 39.3701,33.6181 45.4252,38.4758 51.4803,43.5957 57.5354,48.9778 63.5906,54.6221 69.6457,60.5286 75.7008,66.6973 81.7559,73.1282 87.811,79.8213 93.8661,86.7766 
 99.9213,93.9941 105.976,101.474 112.031,109.216 118.087,117.22 124.142,125.486 130.197,134.014 136.252,142.736 142.307,151.236 148.362,159.514 154.417,167.57 
 160.472,175.404 166.528,183.016 172.583,190.406 178.638,197.575 184.693,204.522 190.748,211.247 196.803,217.75 202.858,224.031 208.913,230.09 214.969,235.927 
 221.024,241.543 227.079,246.937 233.134,252.108 239.189,257.058 245.244,261.786 251.299,266.293 257.354,270.577 263.409,274.64 269.465,278.48 275.52,282.099 
 281.575,285.496 287.63,288.671 293.685,291.624 299.74,294.356 305.795,296.865 311.85,299.153 317.906,301.218 323.961,303.062 330.016,304.684 336.071,306.085 
 342.126,307.263 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mpc 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 time (s) 
 
 
 x2 
 
<polyline clip-path="url(#clip03)" style="stroke:#ff0000; stroke-width:4; stroke-opacity:1; fill:none" points="
 389.37,142.909 395.425,152.501 401.48,162.093 407.535,171.685 413.591,181.278 419.646,190.87 425.701,200.462 431.756,210.055 437.811,219.647 443.866,229.239 
 449.921,238.832 455.976,248.424 462.031,258.016 468.087,267.608 474.142,277.201 480.197,286.793 486.252,293.858 492.307,285.741 498.362,277.624 504.417,269.508 
 510.472,261.391 516.528,253.275 522.583,245.158 528.638,237.042 534.693,228.925 540.748,220.808 546.803,212.692 552.858,204.575 558.913,196.459 564.969,188.342 
 571.024,180.226 577.079,172.109 583.134,163.993 589.189,155.876 595.244,147.759 601.299,139.643 607.354,131.526 613.409,123.41 619.465,115.293 625.52,107.177 
 631.575,99.06 637.63,90.9435 643.685,82.8269 649.74,74.7103 655.795,66.5938 661.85,58.4772 667.906,50.3606 673.961,42.2441 680.016,34.1275 686.071,26.0109 
 692.126,17.8944 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mpc 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 time (s) 
 
 
 u1 
 
<polyline clip-path="url(#clip04)" style="stroke:#ff1493; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 39.3701,380.62 42.4282,380.62 45.4864,380.62 48.5445,380.62 51.6026,380.62 54.6608,380.62 57.7189,380.62 60.7771,380.62 63.8352,380.62 66.8933,380.62 
 69.9515,380.62 73.0096,380.62 76.0678,380.62 79.1259,380.62 82.184,380.62 85.2422,380.62 88.3003,380.62 91.3585,380.62 94.4166,380.62 97.4747,380.62 
 100.533,380.62 103.591,380.62 106.649,380.62 109.707,380.62 112.765,380.62 115.824,380.62 118.882,380.62 121.94,380.62 124.998,380.62 128.056,380.62 
 131.114,380.62 134.172,380.62 137.231,380.62 140.289,380.62 143.347,380.62 146.405,380.62 149.463,380.62 152.521,380.62 155.579,380.62 158.638,380.62 
 161.696,380.62 164.754,380.62 167.812,380.62 170.87,380.62 173.928,380.62 176.986,380.62 180.045,380.62 183.103,380.62 186.161,380.62 189.219,380.62 
 192.277,380.62 195.335,380.62 198.393,380.62 201.452,380.62 204.51,380.62 207.568,380.62 210.626,380.62 213.684,380.62 216.742,380.62 219.8,380.62 
 222.859,380.62 225.917,380.62 228.975,380.62 232.033,380.62 235.091,380.62 238.149,380.62 241.207,380.62 244.265,380.62 247.324,380.62 250.382,380.62 
 253.44,380.62

## Hands-On Example: Bryson Denham
Double integrator problem

### Given:

#### Dynamic Constraints
$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=u_1(t)$$

#### Boundary Conditions
$${x}_1(0)=0 \qquad {x}_1(t_f)=0$$
$${x}_2(0)=1\qquad {x}_2(t_f)=-1$$

#### State Limit
$${x}_{1_{max}}=1/6$$

### Find:
#### The track that minimizes time
$$J=\int_{0}^{1} u(t)^2 dt$$

### Solution: 

## Hands-On Example: Rocket Problem
#### ref: https://github.com/JuliaOpt/juliaopt-notebooks/blob/master/notebooks/JuMP-Rocket.ipynb

### Given:
#### A rocket with drag force losing mass from burning fuel excaping a gravitational feild
#### Dynamic Constraints

$$\dot{x}_1(t)=x_2(t)$$ 
$$\dot{x}_2(t)=(u_1(t)-Drag(t))/x_3(t)-Grav(t)$$
$$\dot{x}_3(t)=-u_1(t)/c$$

with,
$$Drag(t)=D_cx_2(t)^2\exp(-h_c(x_1(t)-h_0)/h_0)$$
$$Grav(t)=g_0(h_0/x_1(t))^2$$

where,
1. $x_1$ is the height
2. $x_2$ is the velocity
3. $x_3$ is the mass
4. $u_1$ is the thrust

#### Boundary Conditions
$${x}_1(0)=h_0\qquad {x}_1(t_f)=free$$
$${x}_2(0)=v_0\qquad {x}_2(t_f)=free$$
$${x}_3(0)=m_0\qquad {x}_3(t_f)=m_f$$

#### State Limits
$${x}_{1_{min}}=h_0\qquad {x}_{1_{max}}=free$$
$${x}_{2_{min}}=v_0\qquad {x}_{1_{max}}=free$$
$${x}_{3_{min}}=m_f\qquad {x}_{1_{max}}=m_0$$

#### Control Limits
$${u}_{1_{min}}=0.0\qquad {u}_{1_{max}}=T_{max}$$

### Find:
#### The track that minimizes rocket height
$$J=x_3(t_f)$$
### Solution: 


In [27]:
# Constants
h_0 = 1.    # Initial height
v_0 = 0.    # Initial velocity
m_0 = 1.    # Initial mass
g_0 = 1.    # Gravity at the surface

# Parameters
T_c = 3.5  # Used for thrust
h_c = 500  # Used for drag
v_c = 620  # Used for drag
m_c = 0.6  # Fraction of initial mass left at end

# Derived parameters
c     = 0.5*sqrt(g_0*h_0)  # Thrust-to-fuel mass
m_f   = m_c*m_0            # Final mass
D_c   = 0.5*v_c*m_0/g_0    # Drag scaling
T_max = T_c*g_0*m_0        # Maximum thrust


Grav=:($g_0*($h_0/x1[j])^2);
Drag=:()  #TODO

de=Array{Expr}(3,);
de[1]=:(x2[j]);
#TODO finish!